In [21]:
# NGSolve Libraries
from ngsPETSc import NonLinearSolver
from netgen.geom2d import unit_square
from ngsolve.solvers import Newton
from ngsolve import *
from ngsolve.krylovspace import GMResSolver
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from scipy.optimize import minimize_scalar
## get problem parameters and geometry
from netgen.geom2d import SplineGeometry

import sys
from IPython.display import clear_output
sys.path.insert(0,"../")
sys.path.insert(0,"../../")
import problems
import numpy as np
import params
import pickle
from time import time

In [22]:


problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.4
ord = 3
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
# def mesher(geom, h):
#     if ".stp" in geom:
#         geo = OCCGeometry(geom)
#     else:
#         geo = pickle.load(open(geom, "rb"))

#     mesh = Mesh(geo.GenerateMesh(maxh=h))
#     return mesh
# mesh = mesher(geom, h)  
w,l= 3,15
n = 5
rect = SplineGeometry()
pnts = [(0,0), (l,0), (l,w), (0,w)]
p1,p2,p3,p4 = [rect.AppendPoint(*pnt,hpref=1) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right"],
        [["line",p3,p4],"top"],
        [["line",p4,p1],"left"]]
rect.Append(curves[0][0], bc = curves[0][1], leftdomain=1, rightdomain=0, maxh=0.1)
rect.Append(curves[1][0], bc = curves[1][1], leftdomain=1, rightdomain=0, maxh=0.1) #0.03
rect.Append(curves[2][0], bc = curves[2][1], leftdomain=1, rightdomain=0, maxh=0.1) #0.05
rect.Append(curves[3][0], bc = curves[3][1], leftdomain=1, rightdomain=0, maxh=0.1) #0.03


mesh = Mesh(rect.GenerateMesh(maxh=0.3)) #0.8
def Gel_energy_functional(F):
    
    g = 9.81
    J = Det(F)
    rho = 1.23e-6
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV #+ rho*g*u[1] - C


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-3)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes, condense=True)






## static condensation

In [23]:

## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((0,0))

psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
obs = CF(-y)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psik,psih = G_funs
    F = Id(2)+ Grad(u)
    if form == "Functional":
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx) # add the "bilinear" form of specific problem
        BF+= delta*v[1]*dx
        BF+= u[1]* vphi*dx
        BF+= -delta*exp(psih)*vphi*dx
        if ord==3:
            BF += -1e-6* delta * vphi*dx
        else: 
            BF += -1e-6* grad(delta) * grad(vphi)*dx
        # the right hand sideBF
        # BF += -alpha * force[2] * v[2] * dx # force is 0 
        BF += -(psik-psih)*v[1]*dx
        BF += -(obs+exp(psih))*vphi*dx
        return BF
    

Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [24]:
cd Proximal_Galerkin

[Errno 2] No such file or directory: 'Proximal_Galerkin'
/Users/sferra/Desktop/NonLInearElasticity/Proximal_Galerkin


/Users/sferra/Library/Python/3.13/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [25]:
max_PG_it = 100
max_iter_newton = 100
max_iter_qnewton = 100
newton_damp = 0.1
softening_n = 100 # 80

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

c = Preconditioner(BF, "bddc") 

gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))
cut = 70
first_bit = gammas[:cut]
second_bit = np.flip(np.linspace(G_target,gammas[cut], 100))
gammas = np.concatenate((first_bit,second_bit))

for num, load in enumerate(gammas):
    print("Softening step", num, "load", load)
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it =60
        tol_newton = 1e-7
        tol_delta = 1e-7
    for k in range(1,max_PG_it):
        print("PG it:",k)
        alpha.Set(1)
        psik.vec.data = psih.vec
        if k != 1:
            uk.vec.data = gfu.components[0].vec 

        G_funs = u,v,alpha,delta,psik, psih

        with TaskManager():
            BF = Assemble_BF(BF,G_funs,form)
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
        print("Solving nonlinear subproblem")
        for it in range(max_iter_qnewton):
            wh =  GridFunction(fesu)
            wh.Set(gfu.components[0])
            print("QN:", it)
            with TaskManager():
                Newton(BF,gfu,maxerr=tol_newton,maxit=max_iter_newton,dampfactor=0.1)

            psih.vec.data = psih.vec + gfu.components[1].vec
            wh_dif = Integrate((gfu.components[0]-wh)*(gfu.components[0]-wh),mesh)
            if wh_dif < tol_QN:
                print("Tol achieved",wh_dif)
                break
        dif = Integrate((uk-gfu.components[0])*(uk-gfu.components[0]),mesh)
        print("U_k difference", dif)
        if dif < tol_delta:
            print("Tol PG achieved", dif)
            break
    clear_output()





Softening step 91 load 0.7000581573308846
PG it: 1
Solving nonlinear subproblem
QN: 0
Newton iteration  0
err =  0.7513656406254635
Newton iteration  1
err =  0.6773890507143038
Newton iteration  2
err =  0.7270547401286797
Newton iteration  3

UMFPACK V5.7.4 (Feb 1, 2016): WARNING: matrix is singular



NgException: UmfpackInverse: Numeric factorization failed.

In [ ]:
cd /Users/sferra/Desktop/NonLInearElasticity/Proximal_Galerkin

In [ ]:
ls

In [26]:
Draw(gfu.components[0], mesh, "u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
gfu_tilde = GridFunction(fesu)
gfu_tilde.vec.data = gfu.components[0].vec
gfu_tilde.components[1].Set(exp(psih)+obs)
u, v = fesu.TnT()
i = BilinearForm(fesu)
i += InnerProduct(u,v)*dx
i.Assemble()
L = LinearForm(fesu)
L += gfu_tilde*v*dx
L.Assemble()
gfu_t = GridFunction(fesu)
gfu_t.vec[:] = 0
gfu_t.vec.data += i.mat.Inverse(freedofs=fesu.FreeDofs())*L.vec
Draw(gfu_t, mesh, "gfu_t")

In [ ]:
Draw(gfu_t-gfu.components[0], mesh, "gfu_t-gfu")

Check proximal point and projected optimization with constraints

In [127]:
import pandas as pd

df = pd.DataFrame()
columns = ["x", "y", "uh_y", "satisfy_constraint", "sum"]
data = []

In [128]:
uh = gfu.components[0]
for i,p in enumerate(mesh.vertices):
    temp = [0,0,0,0,0]
    if p.point[0] == 0:
        temp[0] = p.point[0]
        temp[1] = p.point[1]
        temp[2] = uh(mesh(*p.point))[1]
        temp[3] = (uh(mesh(*p.point))[1]+p.point[1]>0)
        temp[4] = uh(mesh(*p.point))[1]+p.point[1]
        data.append(temp)
df = pd.DataFrame(data, columns=columns)


In [ ]:
pd.set_option('display.max_rows', 500)
df